###### https://intel.github.io/intel-npu-acceleration-library/
###### https://github.com/intel/intel-npu-acceleration-library?tab=readme-ov-file

In [ ]:
!pip install intel-npu-acceleration-library
!pip install transformers

# Run a LLaMA model on the NPU

In [1]:
from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM
import intel_npu_acceleration_library
import torch

In [2]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

model = AutoModelForCausalLM.from_pretrained(model_id, use_cache=True).eval()
tokenizer = AutoTokenizer.from_pretrained(model_id, use_default_system_prompt=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
streamer = TextStreamer(tokenizer, skip_special_tokens=True)

In [3]:
print("Compile model for the NPU")
model = intel_npu_acceleration_library.compile(model, dtype=torch.int8)

Compile model for the NPU


In [4]:
query = input("Ask something: ")
prefix = tokenizer(query, return_tensors="pt")["input_ids"]

In [5]:
generation_kwargs = dict(
    input_ids=prefix,
    streamer=streamer,
    do_sample=True,
    top_k=50,
    top_p=0.9,
    max_new_tokens=512,
)

In [6]:
print("Run inference")
_ = model.generate(**generation_kwargs)

Run inference
안녕하세요! 나는 못할 것이 없죵니다. 혹시 새로 완경을 찾은 나의 아름다움이라는 소리나 그리고 새로운 지역에서 아저씨로부터 이끌어진 것은 싫었다는 말이죵니까? 이들 질문을 한번 생각하는 것은 완벽한 도전이지 않습니까? 실패 때까지도 도전을 해야 할 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x00000237EE8C2A10>>
Traceback (most recent call last):
  File "C:\Program Files\Python311\Lib\site-packages\ipykernel\ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt



# Run KoAlpaca 5.8B - 8bit quantization

In [1]:
from transformers import AutoModelForCausalLM, pipeline
import intel_npu_acceleration_library
import torch

In [ ]:
MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'

model = AutoModelForCausalLM.from_pretrained(
    MODEL, use_cache=True,
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True,
).eval()

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
# model compile
model = intel_npu_acceleration_library.compile(model, dtype=torch.int8)

In [ ]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=MODEL
)

In [ ]:
def ask(x, context='', is_input_full=False):
    ans = pipe(
        f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:",
        do_sample=True,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    print(ans[0]['generated_text'])

In [ ]:
ask("딥러닝이 뭐야?")

In [11]:
ask("소개팅 자리에서 어색하지 않게 대화하고 싶어 혹시 대화 주제를 추천해줄 수 있을까?")

안녕하세요! 저는 중학생입니다. 제가 아는 한에서 몇 가지 소개팅에서 사용할만한 대화 주제를 추천해드릴게요.

1. 취미/취미: 취미를 물어봄으로써 상대방에게 자신의 취향을 언급하는 것도 좋아요.
2. 고향/지역: 자신의 고향이나 지역을 언급하면서 대화를 이어가는 것도 좋아요.
3. 가족/친구: 가족이나 친구에 대한 언급을 하면서 대화를 자연스럽게 이어가는 것도 좋아요.
4. 연인: 연인에 대한 이야기를 하면서 서로의 애칭이나 첫키스 경험 등을 이야기하는 것도 좋아요.
5. 절대/상대: 상대방의 성격이나 외모 등을 평가하는 것도 좋아요.
6. 신청곡: 좋아하는 노래를 언급하면서 추천해달라고 요청하는 것도 좋아요.
7. 공통된 관심사: 서로 관심있는 분야에 대해 이야기하는 것도 좋아요.
8. 자세한 개인 프로필: 개인의 일상생활에 대해 언급하는 것도 좋아요.
9. 미팅: 상대방이 마음에 들 경우에는 당당하게 자신의 생각을 표현하는 것도 좋아요.
10. 연인: 연인에 대한 이야기를 하면서 서로의 애정을 확인하는 것도 좋아요. 

위와 같은 주제들을 추천해드렸지만, 역시 가장 중요한 것은 분위기입니다. 즐거운 대화를 유도하면서 분위기를 좋게 만드는 것이 중요합니다. 말을 쉽지만, 몸에 익히기까지는 많은 노력이 필요합니다. 연습과 실전에서의 경험을 통해 자연스럽게 익히는 것이 가장 좋은 방법입니다. 


In [11]:
script = """
a: 안녕하세요.

b: 안녕하세요.

a: 저는 현우라고 해요. 만나서 반가워요.

b: 반가워요. 저는 지영이에요. 저도 만나서 반가워요. 몇 살이세요?

a: 저는 27살이에요. 지영씨는요?

b: 저는 25살이에요. 어느 대학 다니세요?

a: 저는 연세대에 다니고 있어요. 지영씨는 서울대에 다니고 있으시죠?

b: 네, 맞아요. 혹시 현우씨의 MBTI를 알고 계신가요?

b: 네, 제 MBTI는 INFP에요. 지영씨는요?

a: 저는 INTJ에요. MBTI로 성격을 알아가는 것도 재미있죠. 어쩌면 우리는 서로를 더 잘 이해할 수 있을 거예요. 현우씨는 어떤 취미가 있으세요?

a: 저는 음악 감상을 좋아해요. 특히 재즈 음악을 듣는 걸 좋아해요. 지영씨는요?

b: 저는 피아노 연주를 좋아해요. 그리고 책을 읽는 걸 좋아해요. 현우씨는 어떤 책을 좋아하나요?

a: 주로 소설이나 철학서를 읽어요. 혹시 지영씨가 최근에 읽은 책이 있나요?

b: 네, 최근에는 '무지의 말'이라는 책을 읽었어요. 현우씨는요?

a: 저는 최근에 '인간 본성의 조건'이라는 책을 읽었어요. 지영씨는 책을 어떻게 선택하시나요?

b: 저는 주로 친구의 추천이나 인터넷 리뷰를 참고해서 책을 선택해요. 현우씨는 어떻게 선택하시나요?

a: 저도 비슷해요. 주변 사람들의 추천이나 온라인 리뷰를 참고해서 선택해요. 함께 책을 읽는 것도 재미있을 것 같아요.

b: 좋아요. 그러면 다음 주 토요일에 만나서 함께 책을 읽어볼까요?

a: 좋아요. 그럼 다음 주 토요일에 만나서 함께 좋은 시간 보내요. 그 전에 함께 먹을 식사 메뉴를 정해보는 건 어떨까요?

b: 네. 어떤 음식을 먹을지 함께 정해볼까요?

a: 제가 가봤던 이탈리안 레스토랑이 있는데, 분위기도 좋고 음식도 맛있어요. 그쪽으로 가보는 건 어떠세요?

b: 좋아요! 그럼 그 레스토랑으로 가볼까요?

a: 네, 좋아요. 그럼 지금 출발할까요?

b: 네, 좋아요. 함께 가요.

a: 여기 앉으면서 메뉴를 보면서 대화를 나눠볼까요?

b: 네, 좋아요! 요즘 어떤 일이 있었나요?

a: 요즘에는 회사 일이 바빠서 좀 스트레스 받았어요. 그래도 주말에는 친구들과 모여서 즐거운 시간을 보냈어요. 지영씨는 어떻게 보내셨나요?

b: 저는 요즘에는 취미 생활에 매진하고 있어요. 피아노를 연주하거나 책을 읽는 시간을 많이 가지고 있어요. 특히 피아노를 치면서 마음을 가라앉히고 있어요.

a: 그렇네요. 취미 생활은 정말 마음을 편안하게 해주죠. 피아노 연주를 듣는 건 정말 좋은 시간일 것 같아요.

"""


In [6]:
ask("아래에는 소개팅 자리에서 두 사람이 나누는 대화 목록이 주어질 것이다. 화자 b의 페르소나를 만들고, 특성을 파악하여 화자 a가 던질만한 대화거리를 추천해줘.\n\n\n\n" + script)

대화를 더 잘 이어나가기 위해서는 적절한 질문을 던지는 것이 중요합니다. 아래는 소개팅 자리에서 두 사람이 나눌 수 있는 대화입니다.

- 자기소개를 위한 질문
    - 이름이 무엇인가요?
    - 어디에서 왔나요?
    - 어떻게 일하고 있나요?
    - 전공은 무엇인가요?
    - 좋아하는 과목은 무엇인가요?
    - 최근에 본 책이나 읽은 영화가 있나요?
    - 좋아하는 연예인은 누구인가요?
    - 좋아하는 배드신은 무엇인가요?
    - 가장 좋아하는 TV 프로그램은 무엇인가요?
    - 당신이 살고 있는 지역에서 유명한 음식은 무엇인가요?
    - 당신이 좋아하는 디저트는 무엇인가요?
    - 가장 좋아하는 음료는 무엇인가요?
    - 가장 좋아하는 책은 무엇인가요?
    - 최근에 본 영화가 있나요?
    - 좋아하는 영화가 있나요?
    - 무슨 영화를 가장 좋아하세요?
    - 그 영화에 대해 좀 더 얘기해봐요.

- 취향저격 질문
    - 제가 제일 좋아하는 영화는 반지의 제왕입니다.
    - 좋아하는 영화를 물어보신다면, '타이타닉'을 대답해드릴게요.
    - 최고의 영화 시리즈는 '반지의 제왕'입니다.
    - '타이타닉'은 최고의 로맨틱 영화 중 하나입니다.
    - '반지의 제왕'과 '타이타닉' 중 어느 것을 더 좋아하시나요?
    - '타이타닉'은 1997년 작품으로, 지금까지도 많은 사람들이 인생 최고의 영화로 꼽습니다.
    - 반면, '반지의 제왕'은 2005년 작품으로, 현대적인 스타일로 새롭게 해석되어 많은 사람들에게 사랑을 받았습니다.

- 인터뷰 질문
    - 자기소개를 해주세요.
    - 자신의 취미에 대해 자세히 말해주세요.
    - 자신이 가지고 있는 가장 큰 자랑거리를 알려주세요.
    - 자신이 이룬 가장 큰 성취에 대해 자세하게


In [4]:
from transformers import AutoTokenizer, TextStreamer

alpaca_tokenizer = AutoTokenizer.from_pretrained(MODEL, use_default_system_prompt=True)
alpaca_tokenizer.pad_token_id = alpaca_tokenizer.eos_token_id
alpaca_streamer = TextStreamer(alpaca_tokenizer, skip_special_tokens=True)

def ask(query=""):
    prefix = alpaca_tokenizer(query, return_tensors="pt")["input_ids"]
    prefix.to(device="cuda")

    generation_kwargs = dict(
        input_ids=prefix,
        streamer=alpaca_streamer,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        max_new_tokens=512,
    )

    _ = model.generate(**generation_kwargs)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
ask("딥러닝이 뭐야?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


딥러닝이 뭐야?안녕하세요! 새로운 인공지능 알고리즘에 대해 알고 싶습니다. '알고리즘'이란, 어떠한 문제를 해결하기 위한 단계적... dkking.tistory.com            
### 답변:알고리즘은 수학적인 연산과 논리적인 사고를 필요로 하는 기술입니다. 대표적인 알고리즘으로는 의사결정 알고리즘, 정수 알고리즘, 동적 계획법, 행렬 연산, 미분/적분학, 인공지능, 기계학습 등이 있습니다. 이 중 의사결정 알고리즘은 여러 개의 의사 결정 트리를 만들어 각각의 트리에서 예측한 결과 중에서 가장 많이 선택된 결과를 최종적으로 선택하는 방식입니다. 다른 알고리즘들은 세부적인 조건에 따라 선택의 폭이 달라집니다. 예를 들어, 회귀 분석은 주어진 데이터를 기반으로 어떤 점에서 가장 많이 선택되었는지를 결정하는 반면, 인공 신경망은 더 복잡한 문제를 해결할 수 있습니다. 이러한 알고리즘을 이해하고 활용할 수 있으면, 문제를 해결하는 데 도움이 됩니다. 


In [6]:
ask("소개팅 자리에서 어색하지 않게 대화하고 싶어 혹시 대화 주제를 추천해줄 수 있을까?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


소개팅 자리에서 어색하지 않게 대화하고 싶어 혹시 대화 주제를 추천해줄 수 있을까?
### 맥락:
가장 편하고 좋은 대화 주제는, 
예를 들어, 날씨가 너무 좋다든가,
혹은 최근에 본 영화라든가,
여행을 갔다든가,
그래서 본 영화라든가,
그래서 가보고 싶은 곳이 있다든가,
이렇게 나의 현재 상황과 관련된 것들이 아니겠는가 싶어요. 
호프다이아몬드라는 영화를 보면서, 공감도 많이 했구요.
인생도 그렇지 않을까 하는 생각이 많이 들었습니다.
나의 과거도 그렇고, 나의 현재도 그렇고, 나의 미래도….
사람들은 자신의 과거를 회상할땐, 보통은 부정적인 감정을 많이 느끼게 된다고 합니다.
왜냐하면, 좋은 기억만 기억하고, 나쁜 기억은 잊으려고 하기 때문이겠죠.
하지만, 좋은 기억도 사실은 아픈 기억들이 많거든요.
그런데 아픈 기억은 항상 마음속에 도사리고 있기 마련이죠.
그러니까, 나의 과거를 회상함으로써 나의 마음을 꺼내어
건강한 현재와 미래를 계획할 수 있는 것이랍니다. 
가끔은 살아온 날들을 되돌아 보는 것도 아주 재미있을 거예요.
그리고, 삶속에서 사람들을 만나고, 대화하고, 부대끼다 보면, 
나의 과거뿐 아니라, 나의 미래에 대해서도 알 수 있을 겁니다.
### 답변:대화 주제로는 현재 상황과 관련된 것이 가장 좋습니다. 예를 들어, 날씨가 좋다든가, 영화를 봤다든가, 여행을 갔다든가 하는 것들이 있을 수 있습니다. 그리고 나의 현재 상황과 관련된 것들, 예를 들어, 일을 하고 있지만 만족스럽지 않은 상황 등이 있을 수 있습니다. 
가장 편하고 좋은 대화 주제는 개인적인 것이 될 수도 있습니다. 그리고 호프다이아몬드라는 영화를 보면서 공감하는 부분도 있을 것입니다. 
가끔은 살아온 날들을 되돌아보는 것도 좋을 것입니다. 그리고 삶속에서 사람들을 만나고, 대화하고, 부대끼다 보면, 나의 과거뿐 아니라, 미래에 대해서도 알 수 있을 것입니다. 


# Run KO-Platypus2 7B - 8bit quantization

In [1]:
### KO-Platy
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import intel_npu_acceleration_library
import torch

In [2]:
repo = "kyujinpy/KO-Platypus2-7B-ex"

ko_platypus = AutoModelForCausalLM.from_pretrained(
    repo,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cpu"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
ko_platypus_tokenizer = AutoTokenizer.from_pretrained(repo, use_default_system_prompt=True)
ko_platypus_tokenizer.pad_token_id = ko_platypus_tokenizer.eos_token_id
ko_platypus_streamer = TextStreamer(ko_platypus_tokenizer, skip_special_tokens=True)

In [4]:
ko_platypus = intel_npu_acceleration_library.compile(ko_platypus, dtype=torch.int8)

In [5]:
def ask(query=""):
    prefix = ko_platypus_tokenizer(query, return_tensors="pt")["input_ids"]

    generation_kwargs = dict(
        input_ids=prefix,
        streamer=ko_platypus_streamer,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        max_new_tokens=512,
    )

    _ = ko_platypus.generate(**generation_kwargs)

In [6]:
ask("딥러닝이 뭔지 설명해줘")

딥러닝이뭔지설명해줘.렌 징징 경쟁력을든지 수치렌 지원을 도망 경쟁력을 징징캡캡 Fichier렌 사전 전에경이씨mos 한편 이지씨 든지 coh Przyp 이상의rii좋아든지 이상을 스프 coh좋아외 경쟁력을경이 coh 징징 한편 전에 징징 전에core든지든지 이지재를 도망멧 이상을멧 경쟁력을 coh 이상을 이상의 사전 경쟁력을 사전캡렌 징징렌든지외 개판든지 경쟁력을렌 지원을 경쟁력을씨렌그릇 지원을렌씨 coh씨렌 이상의 도망재를좋아재를 스프 개판 경쟁력을외 도지 경쟁력을그릇외멧 징징 수준의 진행된다 cohrii 경쟁력을 이지 징징 도지 스프 징징참여 가요 수치 이상을좋아 도망 이지 사전 coh 진행된다 전에덩 사전 경쟁력을 도망외 이상의 이상을좋아씨그릇 수치 경쟁력을 coh 경쟁력을 한편 전에외든지 coh 그릇 이상의 전에rii 경쟁력을외좋아그릇참여 전에그릇 이지 사전 이상의경이 이상을mos 이지 한편 이상을 한편경이좋아 경쟁력을 징징든지 경쟁력을 mos경이렌든지방울 경쟁력을 전에 이지참여 이상의 coh재를든지 가요 도망 Selection 수준의든지 coh 이상을든지 전에 이지 지원을 이지 수준의든지 이상을 지원을든지외mos 이상을든지mos 경쟁력을 이지 지원을 사전 징징렌 도지경이 경쟁력을 경쟁력을 이지 도망 경쟁력을 징징든지 전에든지 징징그릇 지원을 경쟁력을 수치 Distribution 이상을 진행된다좋아멧 스프 이상을 사전든지 이지 지원을 이상의 사전 경쟁력을 수준의 이지좋아 징징 Przyp 수치 전에 도망 도망 징징경이렌 징징 이상을외그릇 한편 지원을경이 Selection재를 징징방울방울외 경쟁력을 경쟁력을  경쟁력을그릇 지원을 이상을 전에덩 진행된다방울 이상을 경쟁력을 사전 Przyp 징징 징징 이지 전에 징징 전에 경쟁력을참여 사전 전에외덩멧 징징 수치 coh참여 경쟁력을 사전 스프 징징 경쟁력을 징징,재를 지원을방울 수치 이지 경쟁력을 징징그릇 경쟁력을방울방울 전에 전에 경쟁력을 지원을경이 지원을mos 개판 징징든지든지 사전 경쟁력을 경쟁력을좋아 진행

# Llama-2-Ko-Chat 7B - 8bit quantization

In [1]:
from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM, pipeline
import torch

In [2]:
model_id = "kfkas/Llama-2-ko-7b-Chat"

model = AutoModelForCausalLM.from_pretrained(model_id, use_cache=True).eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
INFERENCE_DEVICE = "cpu"

# Use Intel NPU to accelerate if possible
try:
    import intel_npu_acceleration_library
    print("Compile model for the NPU")
    model = intel_npu_acceleration_library.compile(model, dtype=torch.int8)
except ModuleNotFoundError:
    import sys
    print("Intel NPU Acceleration is disabled on your system.", file=sys.stderr, flush=True)

    # Use CUDA driver
    if torch.cuda.is_available():
        model.to(device="cuda", non_blocking=True)
        print("CUDA Driver is enabled on your system.", flush=True)
        INFERENCE_DEVICE = "cuda"

INFERENCE_DEVICE, model

Compile model for the NPU


('cpu',
 LlamaForCausalLM(
   (model): LlamaModel(
     (embed_tokens): Embedding(46336, 4096, padding_idx=0)
     (layers): ModuleList(
       (0-31): 32 x LlamaDecoderLayer(
         (self_attn): LlamaAttention(
           (rotary_emb): LlamaRotaryEmbedding()
           (kv_proj): QuantizedLinear()
           (q_proj): QuantizedLinear()
           (o_proj): QuantizedLinear()
         )
         (mlp): LlamaMLP(
           (act_fn): SiLU()
           (down_proj): QuantizedLinear()
           (fused_gate_proj_up_proj): QuantizedLinear()
         )
         (input_layernorm): LlamaRMSNorm()
         (post_attention_layernorm): LlamaRMSNorm()
       )
     )
     (norm): LlamaRMSNorm()
   )
   (lm_head): QuantizedLinear()
 ))

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_default_system_prompt=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
streamer = TextStreamer(tokenizer, skip_special_tokens=True)

In [5]:
def ask(query=""):
    prefix = tokenizer(query, return_tensors="pt")["input_ids"]

    generation_kwargs = dict(
        input_ids=prefix,
        streamer=streamer,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        max_new_tokens=512,
    )

    _ = model.generate(**generation_kwargs)

In [3]:
ask("딥러닝이 뭔지 설명해줘")

딥러닝이뭔지설명해줘요!!!!!!!!!!! 58 2009.01.12 12:30 1 0 59 2009.01.12 12:32 1 0 60 2009.01.12 12:36 1 0 61 2009.01.12 12:37 1 0 62 2009.01.12 12:38 1 0 63 2009.01.12 12:38 

KeyboardInterrupt: 

In [6]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    device=INFERENCE_DEVICE,
)

In [7]:
def instruct(user="Who are you?", system="You are a pirate chatbot who always responds in pirate speak!"):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]
    
    prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    outputs = pipe(
        prompt,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    return outputs[0]["generated_text"][len(prompt):]

In [8]:
system_role = "너는 소개팅 자리에서 처음 만난 상대방 역할을 해줘."

In [9]:
instruct("안녕, 저는 김현우라고 합니다. 처음 뵙겠습니다.", system_role)


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



' <<SYS>>\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\

# SKT KoGPT version 2 - 6B

In [14]:
import torch
import intel_npu_acceleration_library
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

model_name = "skt/kogpt2-base-v2"

model = GPT2LMHeadModel.from_pretrained(model_name)
#model = intel_npu_acceleration_library.compile(model, dtype=torch.int8)

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [15]:
text = input("Ask something: ")
input_ids = tokenizer.encode(text, return_tensors="pt")
gen_ids = model.generate(
    input_ids,
    max_length=128,
    repetition_penalty=2.0,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    use_cache=True
)

generated = tokenizer.decode(gen_ids[0])
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


In [16]:
input_ids = tokenizer.encode(script, return_tensors="pt")
gen_ids = model.generate(
    input_ids,
    max_length=1024,
    repetition_penalty=2.0,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    use_cache=True
)

generated = tokenizer.decode(gen_ids[0])
print(generated)


a: 안녕하세요.

b: 안녕하세요.

a: 저는 현우라고 해요. 만나서 반가워요.

b: 반가워요. 저는 지영이에요. 저도 만나서 반가워요. 몇 살이세요?

a: 저는 27살이에요. 지영씨는요?

b: 저는 25살이에요. 어느 대학 다니세요?

a: 저는 연세대에 다니고 있어요. 지영씨는 서울대에 다니고 있으시죠?

b: 네, 맞아요. 혹시 현우씨의 MBTI를 알고 계신가요?

b: 네, 제 MBTI는 INFP에요. 지영씨는요?

a: 저는 INTJ에요. MBTI로 성격을 알아가는 것도 재미있죠. 어쩌면 우리는 서로를 더 잘 이해할 수 있을 거예요. 현우씨는 어떤 취미가 있으세요?

a: 저는 음악 감상을 좋아해요. 특히 재즈 음악을 듣는 걸 좋아해요. 지영씨는요?

b: 저는 피아노 연주를 좋아해요. 그리고 책을 읽는 걸 좋아해요. 현우씨는 어떤 책을 좋아하나요?

a: 주로 소설이나 철학서를 읽어요. 혹시 지영씨가 최근에 읽은 책이 있나요?

b: 네, 최근에는 '무지의 말'이라는 책을 읽었어요. 현우씨는요?

a: 저는 최근에 '인간 본성의 조건'이라는 책을 읽었어요. 지영씨는 책을 어떻게 선택하시나요?

b: 저는 주로 친구의 추천이나 인터넷 리뷰를 참고해서 책을 선택해요. 현우씨는 어떻게 선택하시나요?

a: 저도 비슷해요. 주변 사람들의 추천이나 온라인 리뷰를 참고해서 선택해요. 함께 책을 읽는 것도 재미있을 것 같아요.

b: 좋아요. 그러면 다음 주 토요일에 만나서 함께 책을 읽어볼까요?

a: 좋아요. 그럼 다음 주 토요일에 만나서 함께 좋은 시간 보내요. 그 전에 함께 먹을 식사 메뉴를 정해보는 건 어떨까요?

b: 네. 어떤 음식을 먹을지 함께 정해볼까요?

a: 제가 가봤던 이탈리안 레스토랑이 있는데, 분위기도 좋고 음식도 맛있어요. 그쪽으로 가보는 건 어떠세요?

b: 좋아요! 그럼 그 레스토랑으로 가볼까요?

a: 네, 좋아요. 그럼 지금 출발할까요?

b: 네, 좋아요. 함께 가요.

a: 여기 앉으면서 메뉴를 보면서 대화를 나눠

# KAKAO KoGPT - 1.5B

In [24]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import torch
import intel_npu_acceleration_library
from transformers import AutoTokenizer, AutoModelForCausalLM

repo = "kakaobrain/kogpt"
is_float_32 = False
rev = "KoGPT6B-ryan1.5b" + ("" if is_float_32 else "-float16")

tokenizer = AutoTokenizer.from_pretrained(
    repo, revision=rev,
    bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]',
    pad_token='[PAD]', mask_token='[MASK]'
)
model = AutoModelForCausalLM.from_pretrained(
    repo, revision=rev,
    pad_token_id=tokenizer.eos_token_id,
    low_cpu_mem_usage=True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
model = intel_npu_acceleration_library.compile(model, dtype=torch.int8)

In [3]:
model.to(device=f"cuda", non_blocking=True)
model.eval()

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(64512, 4096)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): QuantizedLinear()
          (v_proj): QuantizedLinear()
          (q_proj): QuantizedLinear()
          (out_proj): QuantizedLinear()
        )
        (mlp): GPTJMLP(
          (fc_in): QuantizedLinear()
          (fc_out): QuantizedLinear()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): QuantizedLinear()
)

In [10]:
prompt = "인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던"

with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').cuda()
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=1024)
    generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [12]:
input_ids = tokenizer.encode(prompt, return_tensors='pt')

gen_ids = model.generate(
    input_ids,
    max_length=1024,
    repetition_penalty=2.0,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    use_cache=True
)

generated = tokenizer.decode(gen_ids[0])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

#### Clear Memory

In [5]:
import gc
torch.cuda.empty_cache()
gc.collect()

2733